<a href="https://colab.research.google.com/github/koojaekwan/Kaggle_study/blob/master/pandas_exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas trick with Titanic data
I refer that Kaggle :
https://www.kaggle.com/python10pm/pandas-100-tricks#Welcome-to-this-Kernel  
* If you fail to load this file in github, please try it!  
https://github.com/iurisegtovich/PyTherm-applied-thermodynamics/issues/11  
https://nbviewer.jupyter.org/  

In [2]:
# basic libraries
import os
import numpy as np
import pandas as pd

In [3]:
# connect to google drive
from google.colab import drive

drive.mount('./MyDrive')

Mounted at ./MyDrive


In [4]:
os.getcwd()   # current working directory
os.chdir('/content/MyDrive/MyDrive')  # setwd like R
os.getcwd() 

'/content/MyDrive/MyDrive'

In [5]:
# this will allow us to print all the files as we generate more in the kernel
def print_files():
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# check Trick 91 for an example
def generate_sample_data(): # creates a fake df for testing
    number_or_rows = 20
    num_cols = 7
    cols = list("ABCDEFG")
    df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
    df.index = pd.util.testing.makeIntIndex(number_or_rows)
    return df

# check Trick 91 for an example
def generate_sample_data_datetime(): # creates a fake df for testing
    number_or_rows = 365*24
    num_cols = 2
    cols = ["sales", "customers"]
    df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
    df.index = pd.util.testing.makeDateIndex(number_or_rows, freq="H")
    return df

# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print_files()

## Trick 89: Split names into first and last name


In [6]:
df = pd.Series(["Geordi La Forge", "Deanna Troi", "Data"]).to_frame()
df.rename({0:"names"}, inplace = True, axis = 1)

df

,names
0,Geordi La Forge
1,Deanna Troi
2,Data


In [7]:
df["names"].str.split(n = 1)

0    [Geordi, La Forge]
1        [Deanna, Troi]
2                [Data]
Name: names, dtype: object

In [8]:
# split on first space  
df["first_name"] = df["names"].str.split(n = 1).str[0] # split 1 times and then take the first one.
df["last_name"] = df["names"].str.split(n = 1).str[1]  # split 1 times and then take the second one.

df

,names,first_name,last_name
0,Geordi La Forge,Geordi,La Forge
1,Deanna Troi,Deanna,Troi
2,Data,Data,NaN


## Trick 88 : Rearrange columns in a df

In [9]:
df = generate_sample_data()

df.head()

/usr/local/lib/python3.6/dist-packages/pandas/util/__init__.py:23: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


,A,B,C,D,E,F,G
0,9,12,7,5,10,3,12
1,8,8,8,11,11,6,14
2,15,2,12,17,18,2,13
3,12,4,12,1,17,1,17
4,8,6,15,2,11,12,10


In [10]:
# Solution 1
df[["A", "C", "D", "F", "E", "G", "B"]].head() # doesn't modify in place

,A,C,D,F,E,G,B
0,9,7,5,3,10,12,12
1,8,8,11,6,11,14,8
2,15,12,17,2,18,13,2
3,12,12,1,1,17,17,4
4,8,15,2,12,11,10,6


In [11]:
# Solution 2
cols_to_move = ["A", "G", "B"]
new_order = cols_to_move + [c for c in df.columns if c not in cols_to_move] 

# generate your new order
df[new_order].head()

,A,G,B,C,D,E,F
0,9,12,12,7,5,10,3
1,8,14,8,8,11,11,6
2,15,13,2,12,17,18,2
3,12,17,4,12,1,17,1
4,8,10,6,15,2,11,12


In [12]:
# Solutin 3: using index
cols = df.columns[[0, 5 , 3, 4, 2, 1, 6]] # df.columns returns a series with index, we use the list to iorder the index as we like --> this way we order the columns

df[cols].head()

,A,F,D,E,C,B,G
0,9,3,5,10,7,12,12
1,8,6,11,11,8,8,14
2,15,2,17,18,12,2,13
3,12,1,1,17,12,4,17
4,8,12,2,11,15,6,10


## Trick 87 : Aggregate you datetime by by and filter weekends

In [13]:
df = generate_sample_data_datetime()

df.shape
df.head()

(8760, 2)

,sales,customers
2000-01-01 00:00:00,2,8
2000-01-01 01:00:00,15,1
2000-01-01 02:00:00,2,6
2000-01-01 03:00:00,14,14
2000-01-01 04:00:00,11,1


In [14]:
df.resample("M").sum() # by Month

,sales,customers
2000-01-31,7398,7414
2000-02-29,6813,7292
2000-03-31,7373,7518
2000-04-30,7296,7399
2000-05-31,7568,7516
2000-06-30,7186,7205
2000-07-31,7557,7607
2000-08-31,7374,7223
2000-09-30,7553,7180
2000-10-31,7509,7271


In [15]:
# Step 1: resample by D. Basically agregate by day and use to_frame() to convert it to frame
daily_sales = df.resample("D")["sales"].sum().to_frame()

daily_sales

,sales
2000-01-01,218
2000-01-02,234
2000-01-03,222
2000-01-04,286
2000-01-05,228
...,...
2000-12-26,235
2000-12-27,277
2000-12-28,250
2000-12-29,219


In [16]:
print(daily_sales.index)
print(daily_sales.index.dayofweek)

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2000-12-21', '2000-12-22', '2000-12-23', '2000-12-24',
               '2000-12-25', '2000-12-26', '2000-12-27', '2000-12-28',
               '2000-12-29', '2000-12-30'],
              dtype='datetime64[ns]', length=365, freq='D')
Int64Index([5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
            ...
            3, 4, 5, 6, 0, 1, 2, 3, 4, 5],
           dtype='int64', length=365)


In [17]:
# Step 2: filter weekends
weekends_sales = daily_sales[daily_sales.index.dayofweek.isin([5, 6])] # saturday, sunday만 취함.

weekends_sales
'''
dayofweek day
0         Monday
1         Tuesday
2         Wednesday
3         Thursday
4         Friday
5         Saturday
6         Sunday
'''

,sales
2000-01-01,218
2000-01-02,234
2000-01-08,229
2000-01-09,228
2000-01-15,253
...,...
2000-12-16,260
2000-12-17,238
2000-12-23,289
2000-12-24,208


'\ndayofweek day\n0         Monday\n1         Tuesday\n2         Wednesday\n3         Thursday\n4         Friday\n5         Saturday\n6         Sunday\n'

## Trick 86 : Named aggregations - avoids multiindex

In [18]:
print_files()

df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# Problem 1
print("The Problem relies on that we don't know the column name")
df.groupby("Pclass")["Age"].agg(["mean", "max"])

The Problem relies on that we don't know the column name


,mean,max
Pclass,,
1,38.233441,80.0
2,29.877630,70.0
3,25.140620,74.0


In [20]:
# Problem 2
print("The Problem relies on that we have multiindex")
df.groupby("Pclass").agg({"Age":["mean", "max"]})

The Problem relies on that we have multiindex


Age      
             mean   max
Pclass                 
1       38.233441  80.0
2       29.877630  70.0
3       25.140620  74.0

In [21]:
# Solution new in pandas 0.25 and higher
print("Now we have solved the previous problems by specifyig the column final name we want.")
print("BUT IT ONLY WORKS WITH A COLUMN. TO THIS KIND OF OPERATIONS ON MULTIPLE COLUMNS CHECK THE NEXT CELL")
df.groupby("Pclass")["Age"].agg(age_mean = "mean", age_max = "max")

Now we have solved the previous problems by specifyig the column final name we want.
BUT IT ONLY WORKS WITH A COLUMN. TO THIS KIND OF OPERATIONS ON MULTIPLE COLUMNS CHECK THE NEXT CELL


,age_mean,age_max
Pclass,,
1,38.233441,80.0
2,29.877630,70.0
3,25.140620,74.0


## Trick 86 : Named aggregations on multiple columns- avoids multiindex

In [22]:
def my_agg(x):
    names = {
        'age_mean': x['Age'].mean(),
        'age_max':  x['Age'].max(),
        'fare_mean': x['Fare'].mean(),
        'fare_max': x['Fare'].max()
    } # define you custom colum names and operations

    return pd.Series(names, index=[ key for key in names.keys()]) # all the columns you create in the previous dictionary will be in this list comprehension

df.groupby('Pclass').apply(my_agg)

# reference
# https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with

,age_mean,age_max,fare_mean,fare_max
Pclass,,,,
1,38.233441,80.0,84.154687,512.3292
2,29.877630,70.0,20.662183,73.5000
3,25.140620,74.0,13.675550,69.5500


## Trick 85 : Convert one type of values to others

In [23]:
# Do some fast feature eng on the DF
d = {"gender":["male", "female", "male"], "color":["red", "green", "blue"], "age":[25, 30, 15]}
df = pd.DataFrame(d)

df

,gender,color,age
0,male,red,25
1,female,green,30
2,male,blue,15


In [24]:
# Solution
map_dict = {"male":"M", "female":"F"}

df["gender_mapped"] = df["gender"].map(map_dict) # using dictionaries to map values

In [25]:
df

,gender,color,age,gender_mapped
0,male,red,25,M
1,female,green,30,F
2,male,blue,15,M


In [26]:
df["color"].factorize()

(array([0, 1, 2]), Index(['red', 'green', 'blue'], dtype='object'))

In [27]:
# using factorize: returns a tuple of arrays (array([0, 1, 2]), 
# Index(['red', 'green', 'blue'], dtype='object')) that's why we select [0]

df["color_factorized"] = df["color"].factorize()[0] 

df

,gender,color,age,gender_mapped,color_factorized
0,male,red,25,M,0
1,female,green,30,F,1
2,male,blue,15,M,2


In [28]:
df["age_compared_boolean"] = df["age"] < 18 # return a True False boolean value

df

,gender,color,age,gender_mapped,color_factorized,age_compared_boolean
0,male,red,25,M,0,False
1,female,green,30,F,1,False
2,male,blue,15,M,2,True


## Trick 84 : Show fewer rows in a df

In [29]:
print("This df occupies way too much space")
df = generate_sample_data()

df

This df occupies way too much space


,A,B,C,D,E,F,G
0,17,4,9,16,1,8,2
1,8,1,7,18,7,4,6
2,2,5,13,10,10,11,14
3,4,12,6,6,9,17,1
4,1,13,2,12,17,15,1
5,17,5,4,12,9,11,4
6,17,18,6,2,10,4,16
7,10,8,9,11,15,13,2
8,7,5,7,13,1,19,13
9,7,7,6,7,9,7,14


In [30]:
print("using set_option to save some screen space")
pd.set_option("display.max_rows", 6)

df

using set_option to save some screen space


,A,B,C,D,E,F,G
0,17,4,9,16,1,8,2
1,8,1,7,18,7,4,6
2,2,5,13,10,10,11,14
...,...,...,...,...,...,...,...
17,6,6,6,1,15,15,14
18,16,4,4,17,1,11,19
19,19,10,6,19,1,8,3


In [31]:
print("use reset_option all to reset to default")
pd.reset_option("all")

df

use reset_option all to reset to default

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/usr/local/lib/python3.6/dist-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


,A,B,C,D,E,F,G
0,17,4,9,16,1,8,2
1,8,1,7,18,7,4,6
2,2,5,13,10,10,11,14
3,4,12,6,6,9,17,1
4,1,13,2,12,17,15,1
5,17,5,4,12,9,11,4
6,17,18,6,2,10,4,16
7,10,8,9,11,15,13,2
8,7,5,7,13,1,19,13
9,7,7,6,7,9,7,14


## Trick 83 : Correct the data types while importing the df

In [32]:
print_files()

df = pd.read_csv("train.csv")

In [33]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
# Step 1: Let's the datetype of the columns
col_types = df.dtypes.to_frame()
col_types

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [35]:
col_types.rename({0:"type"}, inplace = True, axis =1)
col_types

,type
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [36]:
# Step 2: Let's import the previous data and convert it to a dictionary
pd.read_csv("trick83data.csv", index_col = 0)

,type
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [37]:
pd.read_csv("trick83data.csv", index_col = 0)["type"]

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Name: type, dtype: object

In [38]:
col_dict = pd.read_csv("trick83data.csv", index_col = 0)["type"].to_dict()

col_dict

{'Age': 'float64',
 'Cabin': 'object',
 'Embarked': 'object',
 'Fare': 'float64',
 'Name': 'object',
 'Parch': 'int64',
 'PassengerId': 'int64',
 'Pclass': 'int64',
 'Sex': 'object',
 'SibSp': 'int64',
 'Survived': 'int64',
 'Ticket': 'object'}

In [39]:
# Step 3: Edit the dictionary with the correct data types
print("Original dictionary")
col_dict
col_dict["Cabin"] = "category"
col_dict["Embarked"] = "category"
print("Modified dictionary")
col_dict

Original dictionary


{'Age': 'float64',
 'Cabin': 'object',
 'Embarked': 'object',
 'Fare': 'float64',
 'Name': 'object',
 'Parch': 'int64',
 'PassengerId': 'int64',
 'Pclass': 'int64',
 'Sex': 'object',
 'SibSp': 'int64',
 'Survived': 'int64',
 'Ticket': 'object'}

Modified dictionary


{'Age': 'float64',
 'Cabin': 'category',
 'Embarked': 'category',
 'Fare': 'float64',
 'Name': 'object',
 'Parch': 'int64',
 'PassengerId': 'int64',
 'Pclass': 'int64',
 'Sex': 'object',
 'SibSp': 'int64',
 'Survived': 'int64',
 'Ticket': 'object'}

In [40]:
# Step 4: Use the dictionary to import the data
df = pd.read_csv("train.csv", dtype = col_dict)
df.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin          category
Embarked       category
dtype: object

In [41]:
# Note: please note that you can use the dict from step1 and paste in like this
df = pd.read_csv("train.csv", \
                 dtype=
                 {'Age': 'float64',
 'Cabin': 'category',
 'Embarked': 'category',
 'Fare': 'float64',
 'Name': 'object',
 'Parch': 'int64',
 'PassengerId': 'int64',
 'Pclass': 'int64',
 'Sex': 'object',
 'SibSp': 'int64',
 'Survived': 'int64',
 'Ticket': 'object'})

# However, if you have many colums, this can be confusing
df.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin          category
Embarked       category
dtype: object

## Trick 82 : Select data by labe land position (chained iloc and loc)

In [42]:
df = pd.read_csv("train.csv", index_col="PassengerId")
df.iloc[15:20, :].loc[:, "Survived":"Age"] # index는 0부터 시작, 15번째 인덱스는 아이디가 16번째임

,Survived,Pclass,Name,Sex,Age
PassengerId,,,,,
16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0
17,0,3,"Rice, Master. Eugene",male,2.0
18,1,2,"Williams, Mr. Charles Eugene",male,NaN
19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0
20,1,3,"Masselmani, Mrs. Fatima",female,NaN


## Trick 81 : Use apply(type) to see if you have mixed data types

In [43]:
d = {"customer":["A", "B", "C", "D", "E"], "sales":[100, "100", 50, 550.20, "375.25"]}
df = pd.DataFrame(d)

df
# everything seems  but this operation crashes df["sales"].sum(). We have mixed data types
df.dtypes

,customer,sales
0,A,100
1,B,100
2,C,50
3,D,550.2
4,E,375.25


customer    object
sales       object
dtype: object

In [44]:
df["sales"].apply(type) # Wow we can see that we have int, str, floats inn one column

0      <class 'int'>
1      <class 'str'>
2      <class 'int'>
3    <class 'float'>
4      <class 'str'>
Name: sales, dtype: object

In [45]:
df["sales"].apply(type).value_counts() # See the number of each value

<class 'str'>      2
<class 'int'>      2
<class 'float'>    1
Name: sales, dtype: int64

In [46]:
df["sales"] = df["sales"].astype(float) # convert the data to float

print(df["sales"].sum())
print(df["sales"].apply(type).value_counts())

1175.45
<class 'float'>    5
Name: sales, dtype: int64


## Trick 80 : Select multiple slices of columns from a df

In [47]:
df = generate_sample_data().T

df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
A,17,2,17,1,16,10,8,10,16,7,10,5,18,3,9,10,15,16,7,7
B,2,9,3,14,6,4,18,10,17,11,16,17,13,3,18,3,8,14,2,12
C,10,15,13,11,13,5,2,9,13,2,9,17,6,5,8,11,5,13,2,7
D,7,12,9,13,3,14,19,5,5,2,8,3,19,11,18,7,12,14,10,2
E,14,5,16,16,17,6,2,6,15,5,9,15,9,16,3,6,15,12,4,2
F,4,5,17,2,2,4,8,12,3,12,7,9,17,7,15,8,6,1,19,11
G,6,15,14,18,15,6,6,17,1,19,5,16,19,10,18,11,8,2,13,7


In [48]:
cols_str = list(map(str, list(df.columns))) # so that we can do df["0"] as string for the example
df.columns = cols_str

In [49]:
cols_str

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19']

In [50]:
df # column type : str 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
A,17,2,17,1,16,10,8,10,16,7,10,5,18,3,9,10,15,16,7,7
B,2,9,3,14,6,4,18,10,17,11,16,17,13,3,18,3,8,14,2,12
C,10,15,13,11,13,5,2,9,13,2,9,17,6,5,8,11,5,13,2,7
D,7,12,9,13,3,14,19,5,5,2,8,3,19,11,18,7,12,14,10,2
E,14,5,16,16,17,6,2,6,15,5,9,15,9,16,3,6,15,12,4,2
F,4,5,17,2,2,4,8,12,3,12,7,9,17,7,15,8,6,1,19,11
G,6,15,14,18,15,6,6,17,1,19,5,16,19,10,18,11,8,2,13,7


In [51]:
# Using pandas concatenation
# if you are ever confused about axis = 1 or axis = 0, just put axis = "columns" or axis = "rows"
pd.concat([df.loc[:, "0":"2"], df.loc[:, "6":"10"], df.loc[:, "16":"19"]], axis = "columns") 
# ------------------> here we are selecting columns converted to strings

,0,1,2,6,7,8,9,10,16,17,18,19
A,17,2,17,8,10,16,7,10,15,16,7,7
B,2,9,3,18,10,17,11,16,8,14,2,12
C,10,15,13,2,9,13,2,9,5,13,2,7
D,7,12,9,19,5,5,2,8,12,14,10,2
E,14,5,16,2,6,15,5,9,15,12,4,2
F,4,5,17,8,12,3,12,7,6,1,19,11
G,6,15,14,6,17,1,19,5,8,2,13,7


In [52]:
print(list(df.columns[0:3]))
print(np.r_[0:3, 6:11, 16:20])

['0', '1', '2']
[ 0  1  2  6  7  8  9 10 16 17 18 19]


refer : https://rfriend.tistory.com/352

In [53]:
# Using lists
# please ntoe that df.columns is a series with index, so we are using index to filter 
# -------------------------> here we are selecting the index of columns
df[list(df.columns[0:3]) + list(df.columns[6:11]) + list(df.columns[16:20])]

# Using numpy
df.iloc[:, np.r_[0:3, 6:11, 16:20]] # probably the most beautiful solution

,0,1,2,6,7,8,9,10,16,17,18,19
A,17,2,17,8,10,16,7,10,15,16,7,7
B,2,9,3,18,10,17,11,16,8,14,2,12
C,10,15,13,2,9,13,2,9,5,13,2,7
D,7,12,9,19,5,5,2,8,12,14,10,2
E,14,5,16,2,6,15,5,9,15,12,4,2
F,4,5,17,8,12,3,12,7,6,1,19,11
G,6,15,14,6,17,1,19,5,8,2,13,7


,0,1,2,6,7,8,9,10,16,17,18,19
A,17,2,17,8,10,16,7,10,15,16,7,7
B,2,9,3,18,10,17,11,16,8,14,2,12
C,10,15,13,2,9,13,2,9,5,13,2,7
D,7,12,9,19,5,5,2,8,12,14,10,2
E,14,5,16,2,6,15,5,9,15,12,4,2
F,4,5,17,8,12,3,12,7,6,1,19,11
G,6,15,14,6,17,1,19,5,8,2,13,7


## Trick 79 : Count of rows that match a condition

In [54]:
df = generate_sample_data()

df.head()
df.shape

,A,B,C,D,E,F,G
0,15,7,17,9,10,11,10
1,1,10,8,6,11,7,10
2,6,13,3,7,2,13,17
3,7,5,3,7,2,18,12
4,3,10,19,4,11,12,19


(20, 7)

In [55]:
# absolute values
(df["A"] < 5)

0     False
1      True
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9      True
10    False
11    False
12    False
13     True
14    False
15    False
16     True
17    False
18    False
19    False
Name: A, dtype: bool

In [56]:
(df["A"] < 5).sum()

print("\nIn the columns A we have {} of rows that are below 5".format((df["A"] < 5).sum()))

6


In the columns A we have 6 of rows that are below 5


In [57]:
# percentage
(df["A"] < 5).mean()

print("\nIn the columns A the values that are below 5 represent {}%".format((df["A"] < 5).mean()))

0.3


In the columns A the values that are below 5 represent 0.3%


## Trick 78 : Keep track of where your data is coming when you are using multiple sources

In [58]:
# let's generate some fake data
df1 = generate_sample_data()
df2 = generate_sample_data()
df3 = generate_sample_data()
# df1.head()
# df2.head()
# df3.head()
df1.to_csv("trick78data1.csv")
df2.to_csv("trick78data2.csv")
df3.to_csv("trick78data3.csv")

In [59]:
# Step 1 generate list with the file name
lf = []
for _,_, files in os.walk("/content/MyDrive/MyDrive"):
    for f in files:
        if "trick78" in f:
            lf.append(f)
            
lf

['trick78data1.csv', 'trick78data2.csv', 'trick78data3.csv']

In [60]:
# You can use this on your local machine
#from glob import glob
#files = glob("trick78.csv")

# Step 2: assing create a new column named filename and the value is file
# Other than this we are just concatinating the different dataframes
df = pd.concat((pd.read_csv(file).assign(filename = file) for file in lf), ignore_index = True)
df.sample(10)

,Unnamed: 0,A,B,C,D,E,F,G,filename
49,9,17,8,18,18,4,6,3,trick78data3.csv
0,0,11,2,9,14,8,4,10,trick78data1.csv
51,11,10,16,7,19,14,19,7,trick78data3.csv
2,2,14,2,5,5,1,1,15,trick78data1.csv
43,3,11,9,19,8,10,16,3,trick78data3.csv
21,1,11,2,8,3,15,5,13,trick78data2.csv
3,3,1,19,10,9,11,10,16,trick78data1.csv
38,18,13,4,18,18,16,16,14,trick78data2.csv
42,2,4,1,7,8,10,9,2,trick78data3.csv
16,16,19,19,1,11,5,18,12,trick78data1.csv


## Trick 77 : Combine the small categories into a single category names "Others" (using where)

In [61]:
d = {"genre": ["A", "A", "A", "A", "A", "B", "B", "C", "D", "E", "F"]}
df = pd.DataFrame(d)

df
df["genre"].value_counts()

,genre
0,A
1,A
2,A
3,A
4,A
5,B
6,B
7,C
8,D
9,E


A    5
B    2
D    1
C    1
F    1
E    1
Name: genre, dtype: int64

In [62]:
df["genre"].value_counts().nlargest(4)

A    5
B    2
D    1
C    1
Name: genre, dtype: int64

In [63]:
# Step 1: count the frequencies
top_four = df["genre"].value_counts().nlargest(4).index
top_four

Index(['A', 'B', 'D', 'C'], dtype='object')

In [64]:
# Step 2: update the df
df_updated = df.where(df["genre"].isin(top_four), other = "Other")
df_updated["genre"].value_counts()

A        5
B        2
Other    2
D        1
C        1
Name: genre, dtype: int64

## Trick 76 : Filter in pandas only the largest categories

In [65]:
df = pd.read_csv('train.csv')
df.coumns = map(str.lower, list(df.columns)) # convert headers to lower type

df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


(891, 12)

In [66]:
# select top 3 genre
top_genre = df['Embarked'].value_counts().to_frame()[0:2].index

In [67]:
df['Embarked'].value_counts().to_frame()
top_genre

,Embarked
S,644
C,168
Q,77


Index(['S', 'C'], dtype='object')

In [68]:
# now let's filter the df with the top genre
df_top = df[df['Embarked'].isin(top_genre)]
df_top

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [69]:
df_top.shape

(812, 12)

In [70]:
df_top['Embarked'].unique()

array(['S', 'C'], dtype=object)

## Trick 75 : Count the number of words in a pandas series

In [74]:
df = pd.read_csv('train.csv', usecols = ['Name'])

df

,Name
0,"Braund, Mr. Owen Harris"
1,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,"Heikkinen, Miss. Laina"
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,"Allen, Mr. William Henry"
...,...
886,"Montvila, Rev. Juozas"
887,"Graham, Miss. Margaret Edith"
888,"Johnston, Miss. Catherine Helen ""Carrie"""
889,"Behr, Mr. Karl Howell"


In [75]:
df['words'] = df['Name'].str.count(" ") + 1

df

,Name,words
0,"Braund, Mr. Owen Harris",4
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",7
2,"Heikkinen, Miss. Laina",3
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",7
4,"Allen, Mr. William Henry",4
...,...,...
886,"Montvila, Rev. Juozas",3
887,"Graham, Miss. Margaret Edith",4
888,"Johnston, Miss. Catherine Helen ""Carrie""",5
889,"Behr, Mr. Karl Howell",4


## Trick 74 : Webscraping using read_html() and match parameter

* Refer :  
https://towardsdatascience.com/all-pandas-read-html-you-should-know-for-scraping-data-from-html-tables-a3cbb5ce8274  
https://stackoverflow.com/questions/42593104/convert-list-into-a-pandas-data-frame

In [79]:
# Run this on you local machine
url = "https://es.wikipedia.org/wiki/Twitter"
tables = pd.read_html(url)

tables

[                 Twitter, Inc.  ...                                    Twitter, Inc..2
 0                    Editorial  ...                                      Twitter, Inc.
 1                      Creador  ...                                        Jack Dorsey
 2                         Tipo  ...                                    Empresa privada
 3                    Industria  ...                                           Internet
 4                  Forma legal  ...                                             Activo
 5                    Fundación  ...   21 de marzo de 2006 (14 años, 9 meses y 29 días)
 6                     Fundador  ...  Jack Dorsey, Noah Glass, Biz Stone, Evan Williams
 7                         Sede  ...                     San Francisco (Estados Unidos)
 8                   Presidente  ...                                    Omid Kordestani
 9                    Productos  ...                                      microblogging
 10                    Ingresos 

In [86]:
len(tables)

3

In [97]:
matching_tables = pd.read_html(url, match = "Tipo")

matching_tables

[                 Twitter, Inc.  ...                                    Twitter, Inc..2
 0                    Editorial  ...                                      Twitter, Inc.
 1                      Creador  ...                                        Jack Dorsey
 2                         Tipo  ...                                    Empresa privada
 3                    Industria  ...                                           Internet
 4                  Forma legal  ...                                             Activo
 5                    Fundación  ...   21 de marzo de 2006 (14 años, 9 meses y 29 días)
 6                     Fundador  ...  Jack Dorsey, Noah Glass, Biz Stone, Evan Williams
 7                         Sede  ...                     San Francisco (Estados Unidos)
 8                   Presidente  ...                                    Omid Kordestani
 9                    Productos  ...                                      microblogging
 10                    Ingresos 

## Trick 73 : Remove a column and store it as a separate series

In [108]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [109]:
meta = df.pop("Embarked").to_frame()

df.head()
meta.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN


,Embarked
0,S
1,C
2,S
3,S
4,S


## Trick 72 : Convert continuous variable to categorical (cut and qcut)

In [110]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [111]:
# Using cut you can specify the bin edges
pd.cut(df["Age"], bins = [0, 25, 50, 75, 99]).head()

0     (0, 25]
1    (25, 50]
2    (25, 50]
3    (25, 50]
4    (25, 50]
Name: Age, dtype: category
Categories (4, interval[int64]): [(0, 25] < (25, 50] < (50, 75] < (75, 99]]

In [112]:
# Using qcut you can specify the number of bins and it fill generate of aproximate equal size
pd.qcut(df["Age"], q = 3).head()

0    (0.419, 23.0]
1     (34.0, 80.0]
2     (23.0, 34.0]
3     (34.0, 80.0]
4     (34.0, 80.0]
Name: Age, dtype: category
Categories (3, interval[float64]): [(0.419, 23.0] < (23.0, 34.0] < (34.0, 80.0]]

In [113]:
# cut and qcut accept label bin size
pd.qcut(df["Age"], q = 4, labels = ["awful", "bad", "average", "good"]).head()

0        bad
1    average
2        bad
3    average
4    average
Name: Age, dtype: category
Categories (4, object): ['awful' < 'bad' < 'average' < 'good']

## Trick 71 : Read data from a PDF (tabula py)

In [ ]:
# you will have to run on your local machine
#from tabula import read_pdf
# df = read_pdf("test.pdf", pages = "all")

## Trick 70 : Print current version of pandas and it's dependencies

In [114]:
print(pd.__version__)
print(pd.show_versions())

1.1.5


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)



INSTALLED VERSIONS
------------------
commit           : b5958ee1999e9aead1938c0bba2b674378807b3d
python           : 3.6.9.final.0
python-bits      : 64
OS               : Linux
OS-release       : 4.19.112+
Version          : #1 SMP Thu Jul 23 08:00:38 PDT 2020
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.1.5
numpy            : 1.19.5
pytz             : 2018.9
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 51.1.1
Cython           : 0.29.21
pytest           : 3.6.4
hypothesis       : None
sphinx           : 1.8.5
blosc            : None
feather          : 0.4.1
xlsxwriter       : None
lxml.etree       : 4.2.6
html5lib         : 1.0.1
pymysql          : None
psycopg2         : 2.7.6.1 (dt dec pq3 ext lo64)
jinja2           : 2.11.2
IPython          : 5.5.0
pandas_datareader: 0.9.0
bs4              : 4.6.3
bottleneck       : 1